## Import the modules

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as OPT
from torch.utils.data import random_split, DataLoader, TensorDataset
import pandas as pd
import numpy as np

## Taking a look at the Dataset

In [2]:
df = pd.read_csv("../Datasets/HousingData.csv")
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,NaN,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


## Columns (Boston Housing Dataset):

`CRIM`    : The crime rate per town.

`ZN`      : The propotion of the residential zoned over 25.000 sq.ft.

`INDUS`   : The propotion of non-retail business acres per town.

`CHAS`    : Dummy variable.

`NOX`     : The nitric oxides concentration (parts per 10 million).

`RM`      : The average number of rooms per dwelling (κατοικια)

`AGE`     : The age of the resident.

`DIS`     : The distances to five Boston employment centres.

`RAD`     : The index of accessibility to highways.

`TAX`     : The property-tax rate per 10,000(euro).

`PTRATIO` : The student-teacher ratio by town.

`B`       : The proportion of brown people by town.

`LSTAT`   : The lower status of the population.

`MEDV`    : Median value of owner-occupied homes in $1000's.

## Replacing the NaN's values with the mean of each column

In [3]:
numpy_dataset = np.genfromtxt("../Datasets/HousingData.csv", delimiter=',')

mean_per_column = [0 for _ in range(14)]
for row in numpy_dataset[1:]:
    for i, column in enumerate(row):
        if not np.isnan(column):
            mean_per_column[i] += column

mean_per_column = [x / (len(numpy_dataset) - 1) for x in mean_per_column]

inputs = []
outputs = []
for row in numpy_dataset[1:]:
    input_rec = []
    output_rec = []
    for i, column in enumerate(row):
        if np.isnan(column):
            row[i] = mean_per_column[i]
            
        if i != 13:
            input_rec.append(row[i])
        else:
            output_rec.append(row[i])
            
    inputs.append(input_rec)
    outputs.append(output_rec)
    
inputs = torch.tensor(inputs, dtype=torch.float32)
outputs = torch.tensor(outputs, dtype=torch.float32)

## Hyperparameters

In [4]:
input_size = 13
hidden1_size = 32
output_size = 1
training_size = 350
validation_size = 150
test_size = 6
batch_size = 64

## Creating the Data Loaders

In [5]:
train_ds, valid_ds, test_ds = random_split(TensorDataset(inputs, outputs), [training_size, validation_size, test_size])

train_dl = DataLoader(train_ds, batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size)

## Functions to move the Model into the GPU

In [6]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(d, device) for d in data]
    return data.to(device)

class DeviceLoader:
    def __init__(self, data_loader, device):
        self.data_loader = data_loader
        self.device = device
        
    def __iter__(self):
        for batch in self.data_loader:
            yield to_device(batch, self.device)
            
    def __len__(self):
        return len(self.data_loader)

## Moving the Data Loaders into the Device

In [7]:
device = get_default_device()

train_loader = DeviceLoader(train_dl, device)
valid_loader = DeviceLoader(valid_dl, device)
test_loader = DeviceLoader(test_ds, device)

## Creating the Model

In [1]:
class HousingBoston(nn.Module):
    def __init__(self, in_size, h1_size, out_size):
        super().__init__()
        self.linear1 = nn.Linear(in_size, h1_size)
        self.linear2 = nn.Linear(h1_size, out_size)
        
    def __call__(self, input_batch):
        out1 = self.linear1(input_batch)
        out1 = F.leaky_relu(out1)
        out2 = self.linear2(out1)
        out2 = F.leaky_relu(out2)
        model_outputs = F.relu(out2)
        return model_outputs
    
    def training_step(self, batch):
        batch_inputs, batch_outputs = batch
        model_outputs = self(batch_inputs)
        loss = F.mse_loss(model_outputs, batch_outputs)
        return loss
    
    def validation_step(self, batch):
        batch_inputs, batch_outputs = batch
        model_outputs = self(batch_inputs)
        loss = F.mse_loss(model_outputs, batch_outputs)
        return {"valid_batch_loss": torch.sqrt(loss).item()}
    
    def validation_end(self, results):
        avg_loss = torch.tensor([x["valid_batch_loss"] for x in results]).mean().item()
        return {"valid_loss": avg_loss}
    
    def evaluate(self, val_loader):
        valid_results = [self.validation_step(batch) for batch in val_loader]
        return self.validation_end(valid_results)
    
    def epoch_end(self, epoch, results):
        return {"Epoch": epoch+1, "Loss": results["valid_loss"]}
    
    def predict(self, batch):
        batch_inputs, batch_outputs = batch
        model_outputs = self(batch_inputs)
        loss = F.mse_loss(model_outputs, batch_outputs).sqrt()
        return loss.item(), model_outputs.item()

NameError: name 'nn' is not defined

## Creating the Model and moving it to the Device

In [9]:
model = HousingBoston(input_size, hidden1_size, output_size)
model.to(device)

HousingBoston(
  (linear1): Linear(in_features=13, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

## Creating the training loop

In [10]:
def fit(model, epochs, train_loader, valid_loader, opt=OPT.SGD, lr=1e-4):
    history = []
    optimizer = opt(model.parameters(), lr=lr)
    for epoch in range(epochs):
        for train_batch in train_loader:
            loss = model.training_step(train_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        valid_result = model.evaluate(valid_loader)
        epoch_result = model.epoch_end(epoch, valid_result)
        history.append(epoch_result)
        
        if (epoch+1) % 10 == 0:
            print({"Epoch": epoch+1, "Loss": valid_result["valid_loss"]})
            
    return history

In [11]:
history = fit(model, 100, train_loader, valid_loader, lr=1e-6)

{'Epoch': 10, 'Loss': 8.974701881408691}
{'Epoch': 20, 'Loss': 8.345569610595703}
{'Epoch': 30, 'Loss': 8.159552574157715}
{'Epoch': 40, 'Loss': 7.989112854003906}
{'Epoch': 50, 'Loss': 7.897145748138428}
{'Epoch': 60, 'Loss': 7.830381393432617}
{'Epoch': 70, 'Loss': 7.720064163208008}
{'Epoch': 80, 'Loss': 7.672008514404297}
{'Epoch': 90, 'Loss': 7.629024505615234}
{'Epoch': 100, 'Loss': 7.604162693023682}


In [12]:
for batch in test_loader:
    loss, outs = model.predict(batch)
    print(f"Outputs: {batch[1].item()}, Predicted: {outs}, Loss: {loss}")


Outputs: 20.5, Predicted: 28.49228858947754, Loss: 7.992288589477539
Outputs: 23.100000381469727, Predicted: 26.056379318237305, Loss: 2.956378936767578
Outputs: 25.299999237060547, Predicted: 26.29618263244629, Loss: 0.9961833953857422
Outputs: 14.899999618530273, Predicted: 10.933735847473145, Loss: 3.966263771057129
Outputs: 13.800000190734863, Predicted: 17.56223487854004, Loss: 3.762234687805176
Outputs: 8.300000190734863, Predicted: 19.230457305908203, Loss: 10.93045711517334
